In [24]:
import os, sys
from utils import *
from download_DB import download_db
import glob
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

from datetime import date
hoje =  pd.to_datetime(date.today())

In [25]:
files = []
for file in glob.glob("./Queries/v*.txt",):
    a,b = os.path.split(file)
    files.append(b.split(".")[0])
directory = download_db(lista_download=files)


******************** Baixando arquivos *********************

Carregando arquivo de log...
vmonitoramentoleilao já foi baixado hoje: Dia: 10/01/23 - Horário: 13:25:50.  Portanto não foi baixado novamente.
vmonitoramentoug já foi baixado hoje: Dia: 10/01/23 - Horário: 13:28:47.  Portanto não foi baixado novamente.
vmonitoramentousina já foi baixado hoje: Dia: 10/01/23 - Horário: 13:28:52.  Portanto não foi baixado novamente.
vrapeelacesso já foi baixado hoje: Dia: 10/01/23 - Horário: 13:29:03.  Portanto não foi baixado novamente.
vrapeelcontratorecurso já foi baixado hoje: Dia: 10/01/23 - Horário: 13:30:19.  Portanto não foi baixado novamente.
vrapeelcronograma já foi baixado hoje: Dia: 10/01/23 - Horário: 13:31:28.  Portanto não foi baixado novamente.
vrapeelempreendimento já foi baixado hoje: Dia: 10/01/23 - Horário: 13:31:40.  Portanto não foi baixado novamente.
vrapeellicenciamento já foi baixado hoje: Dia: 10/01/23 - Horário: 13:32:20.  Portanto não foi baixado novamente.
vrapeelo

In [26]:
# Lista de colunas que serão usadas várias vezes para a realização de merges entre os dataframes
ide_data_cols = ['DthEnvio','IdeUsinaOutorga']
ide_ug = ['IdeUsinaOutorga','NumOperacaoUg']

# Baixa bancos de dados

In [27]:
# Carrega BD vmonitoramentoleilao => monitoramentoleilao
monitoramentoleilao  = pd.read_parquet(os.path.join(directory,"vmonitoramentoleilao.gzip"))[['IdeUsinaOutorga','CodLeilao', 'DatInicioSuprimento']]
monitoramentoleilao.dropna(subset='DatInicioSuprimento',inplace=True)
monitoramentoleilao.sample(5)

,IdeUsinaOutorga,CodLeilao,DatInicioSuprimento
1038,38081,04/2019,2025-01-01
2131,49403,08/2021,2026-01-01
1449,44619,01/2019,2021-06-28
2590,51440,03/2021,2023-04-01
128,31049,01/2018,2022-01-01


In [28]:
# Carrega BD vrapeelcronograma => rapeel_cronograma
cols = ['IdeUsinaOutorga','DthEnvio','DatRealizacaoII','DatPrevistaAprovacaoIII','DatRealizacaoIII','DatRealizacaoIX','DatRealizacaoXI','DatRealizacaoXIII','DatRealizacaoXIV','DatRealizacaoXII','DatRealizacaoVII','DatRealizacaoVIII','DatRealizacaoVI','DatRealizacaoX','DatRealizacaoV','DatRealizacaoIV']
rename_cols = {
        'DthEnvio' : 'Data_Envio',
        'NomUsina' : 'usina_nome',
        'DatRealizacaoII' : 'canteiroReal',
        'DatPrevistaAprovacaoIII' : 'prev_IO',
        'DatRealizacaoIII' : 'IO_real',
        'DatRealizacaoIX' : 'DesvRio_real',
        'DatRealizacaoXI' : 'Ench_Real',
        'DatRealizacaoXIII' : 'IOTrans_Real',
        'DatRealizacaoXIV' : 'Conc_Trans_REAL',
        'DatRealizacaoXII' : 'Comiss_Real',
        'DatRealizacaoVII' : 'ME_Real_conc_eol',
        'DatRealizacaoVIII' : 'Conc_Combust_Real'}
        
rapeel_cronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].rename(columns=rename_cols)
rapeel_cronograma.sample(5)

,IdeUsinaOutorga,Data_Envio,canteiroReal,prev_IO,IO_real,DesvRio_real,Ench_Real,IOTrans_Real,Conc_Trans_REAL,Comiss_Real,ME_Real_conc_eol,Conc_Combust_Real,DatRealizacaoVI,DatRealizacaoX,DatRealizacaoV,DatRealizacaoIV
36968,38397,2022-11-06 00:03:22.113,NaT,2023-02-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
39921,47461,2023-01-03 13:45:28.163,NaT,2024-10-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1724,35830,2020-05-11 19:26:56.513,NaT,2020-06-26,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
21774,46884,2022-03-02 11:52:28.087,NaT,2022-07-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
28014,54774,2022-07-04 10:11:27.983,NaT,2025-02-10,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [29]:
# Carrega BD vrapeelcontratorecurso => mercado

cols = ['DthEnvio','IdeUsinaOutorga','DatConclusaoIII',]
rename_cols = {'DatConclusaoIII':'DatConclusaoACL'}
mercado = pd.read_parquet(os.path.join(directory,"vrapeelcontratorecurso.gzip"))[cols].rename(columns=rename_cols)

# Carrega BD vrapeellicenciamento => ambiental_datas
cols = ["DatValidadeIV","DatValidadeI","DthEnvio","IdeUsinaOutorga","DatValidadeV"]
rename_cols = {'DatValidadeIV': 'DatValidadeLI','DatValidadeI': 'DatValidadeLP','DatValidadeV': 'DatValidadeLO'}
ambiental_datas = pd.read_parquet(os.path.join(directory,"vrapeellicenciamento.gzip"))[cols].rename(columns=rename_cols)
ambiental_datas = ambiental_datas.loc[ambiental_datas.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]

# Carrega BD vrapeelacesso => acesso_contratos
cols = ["DthEnvio","NomSitContratoI","NomSitContratoIV","NomSituacaoContratoIII","NomSitContratoII","IdeUsinaOutorga"]
rename_cols = {'NomSitContratoI': 'NomSitContratoCCD','NomSitContratoIV': 'NomSitContratoCUST','NomSituacaoContratoIII': 'NomSituacaoContratoCUSD','NomSitContratoII': 'NomSitContratoCCT'}
acesso_contratos = pd.read_parquet(os.path.join(directory,"vrapeelacesso.gzip"))[cols].rename(columns=rename_cols)
acesso_contratos = acesso_contratos.loc[acesso_contratos.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]

# Carrega BD vrapeeloperacaoug => ug_rapeel
cols = ["IdeUsinaOutorga",'NumOperacaoUg',"DthEnvio",'DatPrevistaComercial']
ug_rapeel = pd.read_parquet(os.path.join(directory,"vrapeeloperacaoug.gzip"))[cols]
ug_rapeel = ug_rapeel.loc[ug_rapeel.groupby(['IdeUsinaOutorga','NumOperacaoUg']).DthEnvio.idxmax()]

# Cria dataframe input_rapeel

In [30]:
# Carrega BD vrapeelacesso => acesso_contratos
input_rapeel = pd.merge(mercado,ambiental_datas,on=ide_data_cols,how="inner")
input_rapeel = pd.merge(input_rapeel,acesso_contratos,on=ide_data_cols,how="inner")
input_rapeel = input_rapeel.loc[input_rapeel.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
input_rapeel.sample(5)

,DthEnvio,IdeUsinaOutorga,DatConclusaoACL,DatValidadeLI,DatValidadeLP,DatValidadeLO,NomSitContratoCCD,NomSitContratoCUST,NomSituacaoContratoCUSD,NomSitContratoCCT
598,2022-12-05 14:20:44.827,49651,NaT,NaT,2026-03-10,NaT,Não Assinado,Não se Aplica,Não Assinado,Não se Aplica
937,2023-01-03 09:50:38.467,49820,NaT,NaT,2022-12-30,NaT,Não se Aplica,Válido,Não se Aplica,Não Assinado
1564,2023-01-03 14:32:48.947,48591,NaT,2031-07-12,2031-07-12,2031-07-12,Não se Aplica,Não Assinado,Não se Aplica,Não Assinado
1739,2023-01-03 19:03:08.487,49522,NaT,2030-08-25,2030-08-25,2030-08-25,Não se Aplica,Válido,Não se Aplica,Válido
2742,2023-01-09 13:49:42.220,40848,NaT,2028-09-06,2028-09-06,NaT,Não Assinado,Não Assinado,Não Assinado,Não Assinado


In [31]:
definicoes =   input_rapeel #pd.merge(input_rapeel,ug_rapeel,on=ide_data_cols,how='inner')
dict_validades = {
    'DatValidadeLO' : 'LO',
    'DatValidadeLI' : 'LI',
    'DatValidadeLP' : "LP"
}

definicoes['ValidadeAmbiental'] = pd.NA
for validade in dict_validades:
    definicoes.loc[definicoes.ValidadeAmbiental.isna(),'ValidadeAmbiental'] = definicoes[validade]
definicoes[['IdeUsinaOutorga'] +list(dict_validades.keys()) + ['ValidadeAmbiental']].sample(5)

,IdeUsinaOutorga,DatValidadeLO,DatValidadeLI,DatValidadeLP,ValidadeAmbiental
1596,49172,NaT,2026-02-17,2020-10-05,2026-02-17
973,50810,2031-03-30,2031-03-30,2031-03-30,2031-03-30
1190,35274,NaT,NaT,2019-08-10,2019-08-10
1704,56751,NaT,NaT,2023-03-09,2023-03-09
106,33013,NaT,2021-01-24,2019-05-15,2021-01-24


In [32]:
cols = ["IdeUsinaOutorga","DatCanteiroObraRealizado","DatDesvioRioRealizado","DatEnchimentoRealizado","DatConclusaoSisTransRealizado","DatPrevisaoIniciobra","IdcObraParalisada","IdcUsinaMonitorada","DatMontagemOutorgado","DatConcretagemRealizado","DatConclusaoTorresRealizado","DatInicioObraOutorgado","DatConcretagemOutorgado","DatSisTransmissaoRealizado","IdcSituacaoObra","IdcSemPrevisao","DatComissionamentoUGRealizado","DatMonitoramento","SigTipoGeracao","DscComercializacaoEnergia","DatInicioObraRealizado","DatMontagemRealizado"]

monitoramentodados = pd.read_parquet(os.path.join(directory,"vmonitoramentousina.gzip"))[cols]

In [33]:
#WORK.dadosAcesso t1.'DthEnvio' LABEL="datadainformacao" AS datadainformacao
rename_cols = {'DatPrevistaComercial':'DatPrevistaComercial'}
definicoes_monitoramento = pd.merge(monitoramentodados,definicoes,on='IdeUsinaOutorga',how="left").rename(columns=rename_cols).drop_duplicates()
definicoes_monitoramento = definicoes_monitoramento.loc[definicoes_monitoramento.IdcUsinaMonitorada == "Sim"]

definicoes_monitoramento['CondicaoAmbiental'] = pd.NA

for validade in reversed(dict_validades):
    definicoes_monitoramento.loc[definicoes_monitoramento[validade].notna(),'CondicaoAmbiental'] = dict_validades[validade]

definicoes_monitoramento.loc[(definicoes_monitoramento.CondicaoAmbiental.isna()) & (definicoes_monitoramento.DthEnvio.notna()),'CondicaoAmbiental'] = 'Sem LP'
definicoes_monitoramento.loc[(definicoes_monitoramento.CondicaoAmbiental.isna()),'CondicaoAmbiental'] = 'Não Informado'


definicoes_monitoramento[list(dict_validades) + ['ValidadeAmbiental','CondicaoAmbiental']].sample(10)

display(definicoes_monitoramento[list(dict_validades.keys()) + ['CondicaoAmbiental','ValidadeAmbiental']])

definicoes_monitoramento['CondicaoConexao'] = "Não informado"

nsa = "Não se Aplica"
nass = "Não Assinado"
valido = "Válido"
vencido = "Vencido"

definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSitContratoCCD ==  nsa)&
    (definicoes_monitoramento.NomSitContratoCCT == nsa) &
    definicoes_monitoramento.DthEnvio.notna()
    )
    ,'CondicaoConexao'] = "Verificar"

definicoes_monitoramento.loc[
(    ((definicoes_monitoramento.NomSitContratoCCD ==  valido) |
    (definicoes_monitoramento.NomSitContratoCCT == valido)) &
    definicoes_monitoramento.DthEnvio.notna())
    
    ,'CondicaoConexao'] = "OK"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.NomSitContratoCCD ==  vencido) |
    (definicoes_monitoramento.NomSitContratoCCT == vencido) |
    (definicoes_monitoramento.NomSitContratoCCD == nass) |
    (definicoes_monitoramento.NomSitContratoCCT == nass )) &
    definicoes_monitoramento.DthEnvio.notna())
    ,'CondicaoConexao'] = "Sem Conexão"
display(definicoes_monitoramento[['IdeUsinaOutorga'] + ['NomSitContratoCCD','NomSitContratoCCT','DthEnvio','CondicaoConexao']].sample(10))


definicoes_monitoramento['CondicaoUso'] = "Não informado"

definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSituacaoContratoCUSD ==  nsa)&
    (definicoes_monitoramento.NomSitContratoCUST == nsa) 
    )
    ,'CondicaoUso'] = "Verificar"


definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSituacaoContratoCUSD ==  valido) |
    (definicoes_monitoramento.NomSitContratoCUST == valido) 
    )
    ,'CondicaoUso'] = "OK"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.NomSituacaoContratoCUSD ==  vencido) |
    (definicoes_monitoramento.NomSitContratoCUST == vencido) |
    (definicoes_monitoramento.NomSituacaoContratoCUSD == nass) |
    (definicoes_monitoramento.NomSitContratoCUST == nass )))
    ,'CondicaoUso'] = "Sem Uso"

display(definicoes_monitoramento[['IdeUsinaOutorga'] + ['NomSituacaoContratoCUSD','NomSitContratoCUST','CondicaoConexao']].sample(10))


definicoes_monitoramento["PPA"] = "Ambos"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "ACR") & 
        definicoes_monitoramento.DatConclusaoACL.isna())),
    "PPA"] = "ACR"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "Fora do ACR") & 
        definicoes_monitoramento.DatConclusaoACL.isna())),
    "PPA"] = "Nenhum"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "Fora do ACR") & 
        definicoes_monitoramento.DatConclusaoACL.notna())),
    "PPA"] = "ACL"

display(definicoes_monitoramento[['IdeUsinaOutorga'] + ['DscComercializacaoEnergia','DatConclusaoACL','PPA']].sample(10))

,DatValidadeLO,DatValidadeLI,DatValidadeLP,CondicaoAmbiental,ValidadeAmbiental
0,NaT,2020-12-11,2017-09-18,LI,2020-12-11
1,NaT,2026-03-09,2002-12-11,LI,2026-03-09
2,NaT,NaT,NaT,Sem LP,NaT
3,2026-02-22,2028-06-28,2000-03-29,LO,2026-02-22
7,NaT,NaT,NaT,Não Informado,NaT
...,...,...,...,...,...
3093,NaT,NaT,NaT,Não Informado,NaT
3094,NaT,NaT,NaT,Não Informado,NaT
3095,NaT,NaT,NaT,Não Informado,NaT
3096,NaT,NaT,NaT,Não Informado,NaT


,IdeUsinaOutorga,NomSitContratoCCD,NomSitContratoCCT,DthEnvio,CondicaoConexao
362,33833,Não se Aplica,Válido,2023-01-04 10:58:43.917,OK
2493,50809,Não se Aplica,Não Assinado,2023-01-02 10:15:30.687,Sem Conexão
40,28386,Não se Aplica,Não se Aplica,2022-04-13 10:37:03.257,Verificar
102,30541,Válido,Não se Aplica,2023-01-04 14:17:11.480,OK
652,35782,Válido,Não se Aplica,2023-01-03 11:41:50.217,OK
2889,54625,Não se Aplica,Não Assinado,2023-01-02 09:43:08.680,Sem Conexão
2315,49840,Não se Aplica,Não Assinado,2023-01-03 11:57:13.367,Sem Conexão
2875,54553,NaN,NaN,NaT,Não informado
2389,50042,Não se Aplica,Válido,2023-01-02 14:31:45.273,OK
2972,54914,Não se Aplica,Não se Aplica,2023-01-03 17:13:59.220,Verificar


,IdeUsinaOutorga,NomSituacaoContratoCUSD,NomSitContratoCUST,CondicaoConexao
1019,38059,Não se Aplica,Não Assinado,Sem Conexão
1346,44413,NaN,NaN,Não informado
1482,45026,Válido,Não se Aplica,OK
2247,49654,Não Assinado,Não se Aplica,Sem Conexão
228,32404,Não se Aplica,Válido,OK
466,34612,Válido,Não se Aplica,Sem Conexão
3085,70550,NaN,NaN,Não informado
1615,46702,Não se Aplica,Não se Aplica,Verificar
1579,46529,Não se Aplica,Válido,OK
2998,55379,NaN,NaN,Não informado


,IdeUsinaOutorga,DscComercializacaoEnergia,DatConclusaoACL,PPA
1580,46530,Fora do ACR,NaT,Nenhum
2819,53878,Fora do ACR,NaT,Nenhum
2269,49693,ACR,NaT,ACR
2890,54626,Fora do ACR,NaT,Nenhum
2677,51984,Fora do ACR,NaT,Nenhum
2810,53869,Fora do ACR,NaT,Nenhum
2407,50112,Fora do ACR,NaT,Nenhum
2224,49622,Fora do ACR,NaT,Nenhum
2448,50473,Fora do ACR,NaT,Nenhum
2330,49890,Fora do ACR,NaT,Nenhum


In [34]:
# Apenas mudança de nome
tabela_criterio = definicoes_monitoramento

In [35]:
ea = "Em andamento"
na = "Não Iniciada"
nenhum = "Nenhum"
paralisada = "Paralisada"
ok = "OK"

tabela_criterio['criterio'] = np.select(
    [
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum))
    ],
    [0,1,2,3,4,5,8,7],
    default=6
)


tabela_criterio['dscriterio'] = np.select(
    [
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum))
    ],
    [
    "Usina em Obras em andamento",
    "Sem obras e PPA Ok",
    "Sem obras, sem PPA e LI Ok e Cust OK",
    "Sem obras, sem PPA e LI OK e Cust não Ok",
    "Sem obras, sem PPA e LP OK e Cust Ok",
    "Sem obras, sem PPA e LP OK e Cust não Ok",
    "Obras paralisadas",
    "Sem obras, sem PPA e sem Rapeel"
],
    default="Sem LP")

display(tabela_criterio[['IdeUsinaOutorga','IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio','dscriterio']].sample(10))

,IdeUsinaOutorga,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio,dscriterio
2917,54653,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
2713,52257,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
3092,70557,Não Iniciada,Nenhum,Não Informado,Não informado,7,"Sem obras, sem PPA e sem Rapeel"
1557,46097,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
1368,44447,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
2617,51641,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
1265,42999,Em andamento,ACL,LO,OK,0,Usina em Obras em andamento
1561,46509,Não Iniciada,Nenhum,LI,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
2358,49940,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
1297,43208,Não Iniciada,Nenhum,LI,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"


In [36]:
slp = "Sem LP"
sim = "Sim"
tabela_criterio['criterio_novo'] = np.select(
    [
        (tabela_criterio.IdcSemPrevisao == sim),
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        (tabela_criterio.CondicaoAmbiental == "Sem LP"),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum)),
        
    ],
    [9,0,1,2,4,3,5,8,6,7],
    default=-8
)


display(tabela_criterio[['IdeUsinaOutorga','IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio_novo','dscriterio']].sample(5))

,IdeUsinaOutorga,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio_novo,dscriterio
1882,47455,Não Iniciada,Nenhum,LO,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
1203,40816,Em andamento,ACL,LO,OK,0,Usina em Obras em andamento
3075,70540,Não Iniciada,Nenhum,Não Informado,Não informado,7,"Sem obras, sem PPA e sem Rapeel"
2265,49689,Em andamento,ACR,LI,OK,0,Usina em Obras em andamento
2511,50867,Não Iniciada,ACL,LP,OK,1,Sem obras e PPA Ok


In [37]:
previsao_path = os.path.join(get_standard_folder_path("Documents"),r"Previsor\Previsoes")
previsao_file = sorted(glob.glob(f"{previsao_path}/*.gzip"))[-1]
calculo_previsao = pd.read_parquet(previsao_file)
calculo_previsao.FaseAtual =calculo_previsao.FaseAtual.str.slice(0,-3)
calculo_previsao.Indicador =calculo_previsao.Indicador/100


In [38]:
cols = ["Dat_OC_obrigacao","DatPrevisaoIniciobra","FASE","Ind_crono_norm","IdeUsinaOutorga","flagOPTeste30dias",'Previsao_OC']
rename_cols = {'FaseAtual':'FASE','Indicador':'Ind_crono_norm'}
calculo_previsao = calculo_previsao.rename(columns=rename_cols)

for col in ['SigTipoGeracao','DatMonitoramento ']:
    if 'SigTipoGeracao' in calculo_previsao.columns:
        calculo_previsao.drop(['SigTipoGeracao'],inplace=True,axis=1)

### criterios_da_previsao

In [45]:
criterios_da_previsao  = pd.merge(
                                    tabela_criterio,calculo_previsao,
                                    on="IdeUsinaOutorga",how='inner'
                                    )
criterios_da_previsao = pd.merge(criterios_da_previsao,ug_rapeel,left_on= ['IdeUsinaOutorga','NumUgUsina'],right_on=['IdeUsinaOutorga','NumOperacaoUg'],how='left')

In [47]:
################################ regra_antiga ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'regra_antiga'] = pd.NA
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'regra_antiga'] = criterios_da_previsao[['Previsao_OC','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'regra_antiga'] = ( hoje +pd.Timedelta(60,'D'))
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'regra_antiga'] = criterios_da_previsao['Previsao_OC']
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Sim")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC']].max(axis=1)
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] >= 3) & ( (criterios_da_previsao['IdcObraParalisada'] == "Sim")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[criterios_da_previsao['criterio'] >= 3 & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao']].max(axis=1)


################################ regrapmo ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'regrapmo'] =  pd.NA
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC']].max(axis=1)
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'regrapmo'] = ( hoje + pd.Timedelta(60,'D'))
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'regrapmo'] = criterios_da_previsao['Previsao_OC']
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 8) ),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC']].max(axis=1)

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(3*365,'D')
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 3),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(4*365,'D')
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 4),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(5*365,'D')
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 5) | (criterios_da_previsao['criterio'] == 7),'regrapmo'] =  criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 6),'regrapmo'] = pd.NA

In [48]:
################################ dscjustificativapmo ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'dscjustificativapmo'] = "Analisar justificativa"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT") & ( criterios_da_previsao['Previsao_OC'] > criterios_da_previsao['DatPrevistaComercial']),'dscjustificativapmo'] = "Estágio Atual das Obras"
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT") & ( criterios_da_previsao['Previsao_OC'] < criterios_da_previsao['DatPrevistaComercial']),'dscjustificativapmo'] = "Estágio Atual das Obras"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'dscjustificativapmo'] = "Usina liberada para testes"
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'dscjustificativapmo'] = "Testes não concluídos"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 8)) & ( criterios_da_previsao['DatPrevistaComercial'] > criterios_da_previsao['Dat_OC_obrigacao']),'dscjustificativapmo'] = "Paralisação de Obras"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 8)) & ( criterios_da_previsao['DatPrevistaComercial'] < criterios_da_previsao['Dat_OC_obrigacao']),'dscjustificativapmo'] = "Paralisação de Obras"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")) & ( criterios_da_previsao['DatPrevistaComercial'] > criterios_da_previsao['Previsao_OC']),'dscjustificativapmo'] = "Estágio Atual das Obras"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")) & ( criterios_da_previsao['DatPrevistaComercial'] < criterios_da_previsao['Previsao_OC']),'dscjustificativapmo'] = "Estágio Atual das Obras" 
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 3),'dscjustificativapmo'] = "Não celebrado contrato de uso do sistema de transmissão/distribuição"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 4) | (criterios_da_previsao['criterio'] == 5),'dscjustificativapmo'] = "Situação da Licença de Instalação"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 7),'dscjustificativapmo'] = "Sem Rapeel"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 6),'dscjustificativapmo'] = "Sem Licenciamento Ambiental"


################################ criterionovopmo ################################
criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'criterionovopmo'] = 0.1

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'criterionovopmo'] = 0.2

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'criterionovopmo'] = 0.3

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 1),'criterionovopmo'] = 1

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 2),'criterionovopmo'] = 2

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 3),'criterionovopmo'] = 3

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 5),'criterionovopmo'] = 5

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 4),'criterionovopmo'] = 4

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 6),'criterionovopmo'] = 6

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 7),'criterionovopmo'] = 7

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 8),'criterionovopmo'] = 8

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 9),'criterionovopmo'] = 9

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 9),'dsccriterionovo'] = "Usina Viabilidade Baixa análise da fiscalização"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'dsccriterionovo'] = "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'dsccriterionovo'] = "Usina em Teste há mais de 30 dias = Previsão OC próximos 60 dias"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'dsccriterionovo'] = "Usina em Teste = Previsão OC conforme data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 1),'dsccriterionovo'] = "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 2),'dsccriterionovo'] = "Usina sem obras com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 3),'dsccriterionovo'] = "Usina sem obras, sem cust, sem PPA e com LI = Previsão OC handicap de 5 anos"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 4),'dsccriterionovo'] = "Usina sem obras, com cust, sem PPA e sem LI = Previsão OC handicap de 5 anos"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 5),'dsccriterionovo'] = "Usina sem obras, sem cust, sem PPA e sem LI = Previsão OC handicap de 5 anos"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 7),'dsccriterionovo'] = "Usina sem obras, sem PPA e sem RAPEEL = Previsão OC handicap de 5 anos"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 6),'dsccriterionovo'] = "Usina sem LP = Previsão OC sem previsão"
criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 8),'dsccriterionovo'] = "Usina obras paralisadas == Previsão OC maior entre data RAPEEL e data compromisso"

In [49]:
################################ regranovapmo ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'regranovapmo'] = pd.NA

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC']].max(axis=1)

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'regranovapmo'] = ( hoje + pd.Timedelta(60,"D"))

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['Previsao_OC'].isna()),'regranovapmo'] = criterios_da_previsao['DatPrevistaComercial']

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['Previsao_OC'].notna()),'regranovapmo'] = criterios_da_previsao['Previsao_OC']

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo']== 1) | (criterios_da_previsao['criterio_novo']== 2) ,'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(5*365,'D')

criterios_da_previsao.loc[((criterios_da_previsao['criterio_novo'] == 3) | (criterios_da_previsao['criterio_novo'] == 4) | (criterios_da_previsao['criterio_novo'] == 5) ),'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)

criterios_da_previsao.loc[criterios_da_previsao['criterio_novo']== 6,'regranovapmo'] = pd.NA

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(6*365,'D')
criterios_da_previsao.loc[criterios_da_previsao['criterio_novo']== 7,'regranovapmo'] =  criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)

criterios_da_previsao.loc[criterios_da_previsao['criterio_novo']== 8,'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)


################################ criterio_pmo_ts ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'criterio_pmo_ts'] = 8

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'criterio_pmo_ts'] = 0.1

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'criterio_pmo_ts'] = 0.2

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'criterio_pmo_ts'] = 0.3

criterios_da_previsao.loc[((criterios_da_previsao['PPA'] != "Nenhum")) & ((criterios_da_previsao['criterio'] == 8) ),'criterio_pmo_ts'] = 1.1

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'criterio_pmo_ts'] = 1.2

criterios_da_previsao.loc[((criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'criterio_pmo_ts'] = 2.2

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 8)) & ( (criterios_da_previsao['PPA'] == "Nenhum") & (criterios_da_previsao['CondicaoUso'] == "OK")) & ( (criterios_da_previsao['CondicaoAmbiental'] == "LI") | (criterios_da_previsao['CondicaoAmbiental'] == "LO") ),'criterio_pmo_ts'] = 2.1

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 3),'criterio_pmo_ts'] = 3

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 4),'criterio_pmo_ts'] = 4

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 5),'criterio_pmo_ts'] = 5

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 7),'criterio_pmo_ts'] = 7

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 6),'criterio_pmo_ts'] = 6

In [50]:
################################ dsccriterio_pmo_ts ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'dsccriterio_pmo_ts'] = "Usina Viabilidade Baixa análise da fiscalização"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'dsccriterio_pmo_ts'] = "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'dsccriterio_pmo_ts'] = "Usina em Teste há mais de 30 dias == Previsão OC próximos 60 dias"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'dsccriterio_pmo_ts'] = "Usina em Teste == Previsão OC conforme data calculada"

criterios_da_previsao.loc[((criterios_da_previsao['PPA'] != "Nenhum")) & ((criterios_da_previsao['criterio'] == 8) ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas com PPA = Previsão OC maior entre data RAPEEL e data compromisso"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'dsccriterio_pmo_ts'] = "Usina Viabilidade Baixa análise da fiscalização"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'dsccriterio_pmo_ts'] = "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'dsccriterio_pmo_ts'] = "Usina em Teste há mais de 30 dias = Previsão OC próximos 60 dias"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'dsccriterio_pmo_ts'] = "Usina em Teste = Previsão OC conforme data calculada"

criterios_da_previsao.loc[((criterios_da_previsao['PPA'] != "Nenhum")) & ((criterios_da_previsao['criterio'] == 8) ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas com PPA = Previsão OC maior entre data RAPEEL e data compromisso"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[((criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras sem PPA com LI com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 8)) & ( (criterios_da_previsao['PPA'] == "Nenhum") & (criterios_da_previsao['CondicaoUso'] == "OK")) & ( (criterios_da_previsao['CondicaoAmbiental'] == "LI") | (criterios_da_previsao['CondicaoAmbiental'] == "LO") ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas sem PPA com LI com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 3),'dsccriterio_pmo_ts'] = "Usina sem PPA com LI sem CUST = Previsão OC maior entre handicap de 3 anos e data RAPEEL e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 4),'dsccriterio_pmo_ts'] = "Usina sem PPA sem LI com CUST = Previsão OC maior entre handicap de 4 anos e data RAPEEL e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 5),'dsccriterio_pmo_ts'] = "Usina sem PPA sem LI sem CUST = Previsão OC maior entre handicap de 5 anos e data RAPEEL e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 7),'dsccriterio_pmo_ts'] = "Usina sem obras, sem PPA e sem RAPEEL = Previsão OC maior entre handicap de 5 anos e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 6),'dsccriterio_pmo_ts'] = "Usina sem LP = Previsão OC sem previsão"

In [51]:
################################ criterio_pmo_ts_max_usina ################################
criterio_pmo_ts_max = criterios_da_previsao.groupby("IdeUsinaOutorga")[['criterio_pmo_ts']].max().rename(columns=
{'criterio_pmo_ts': 'criterio_pmo_ts_max_usina'})
criterios_da_previsao = criterios_da_previsao.merge(criterio_pmo_ts_max,on="IdeUsinaOutorga",how='left')


################################ dscjustificativaregra ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'dscjustificativaregra'] = "Analisar justificativa: Revogação da outorga em avaliação ou Demandas judiciais ou Inviabilidade da implantação da usina"

criterios_da_previsao.loc[(criterios_da_previsao['IdcObraParalisada'] == "Sim") & (criterios_da_previsao['IdcSemPrevisao'] != "Sim"),'dscjustificativaregra'] = "Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 0.2),'dscjustificativaregra'] = "Analisar justificativa: Sem Licença de Operação (LO) ou Alterações de características técnicas ou Estágio das obras da conexão associada ou Demandas judiciais ou Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 0.1) | (criterios_da_previsao['criterio_pmo_ts'] == 0.3),'dscjustificativaregra'] = "Estágio das obras da usina"


criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 1.1),'dscjustificativaregra'] = "Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 1.2),'dscjustificativaregra'] = "Compromisso de venda de energia - PPA"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 2.2),'dscjustificativaregra'] = "Licença de Instalação - LI e acesso a rede - CUST/CUSD"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 3),'dscjustificativaregra'] = "Acesso a rede contratado - CUST/CUSD"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 4),'dscjustificativaregra'] = "Acesso a rede não contratado - CUST/CUSD"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 5),'dscjustificativaregra'] = "Sem Licença de Instalação - LI"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 6),'dscjustificativaregra'] = "Nenhuma licença ambiental válida"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 7),'dscjustificativaregra'] = "Sem RAPEEL"


################################ dscjustificativaregranova ################################
criterios_da_previsao.loc[(criterios_da_previsao['criterionovopmo'] == 9),'dscjustificativaregranova'] = "Analisar justificativa: Revogação da outorga em avaliação ou Demandas judiciais ou Inviabilidade da implantação da usina"

criterios_da_previsao.loc[(criterios_da_previsao['criterionovopmo'] == 8),'dscjustificativaregranova'] = "Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao['criterionovopmo'] == 0.2),'dscjustificativaregranova'] = "Analisar justificativa: Sem Licença de Operação (LO) ou Alterações de características técnicas ou Estágio das obras da conexão associada ou Demandas judiciais ou Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao['criterionovopmo'] == 0.1) | (criterios_da_previsao['criterio_pmo_ts'] == 0.3),'dscjustificativaregranova'] = "Estágio das obras da usina"

criterios_da_previsao.loc[(criterios_da_previsao['criterionovopmo'] == 1),'dscjustificativaregranova'] = "Compromisso de venda de energia - PPA"

criterios_da_previsao.loc[criterios_da_previsao['criterionovopmo']==2,'dscjustificativaregranova'] = "Acesso a rede contratado - CUST/CUSD"

criterios_da_previsao.loc[criterios_da_previsao['criterionovopmo']==3,'dscjustificativaregranova'] = "Acesso a rede não contratado - CUST/CUSD"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 4),'dscjustificativaregranova'] = "Sem Licença de Instalação - LI"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 5),'dscjustificativaregranova'] = "Sem Licença de Instalação - LI"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 6),'dscjustificativaregranova'] = "Nenhuma licença ambiental válida"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 7),'dscjustificativaregranova'] = "Sem RAPEEL"

In [55]:
criterios_da_previsao_sas = pd.read_clipboard()

C:\Users\viniciusbarros\AppData\Local\Temp\ipykernel_10964\4006681696.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  criterios_da_previsao_sas = pd.read_clipboard()


In [65]:
criterios_da_previsao_sas.shape

(73672, 9)

In [73]:
criterios_da_previsao_sas['SAS'] = True

In [74]:
criterios_da_previsao['PY'] = True

In [66]:
criterios_da_previsao.shape

(73683, 64)

In [81]:
criterios_da_previsao_sas[criterios_da_previsao_sas.IdeUsinaOutorga == 51589]

,IdeUsinaOutorga,NumUgUsina,CriterioNovoPMO,DscCriterioNovo,RegraNovaPMO,CriterioPMO_TS,DscCriterioPMO_TS,DscJustificativaRegra,DscJustificativaRegranova,SAS
19311,51589,1,1,Usina sem obras com PPA = Previsão OC maior en...,25Jun2024,"1,2",Usina sem obras com PPA = Previsão OC maior en...,Compromisso de venda de energia - PPA,Compromisso de venda de energia - PPA,True


In [75]:
merge = pd.merge(criterios_da_previsao,criterios_da_previsao_sas,on=['IdeUsinaOutorga','NumUgUsina'],how='outer')

In [77]:
merge[merge.PY.isna() & merge.SAS.notna()]

,IdeUsinaOutorga,DatCanteiroObraRealizado,DatDesvioRioRealizado,DatEnchimentoRealizado,DatConclusaoSisTransRealizado,DatPrevisaoIniciobra_x,IdcObraParalisada,IdcUsinaMonitorada,DatMontagemOutorgado,DatConcretagemRealizado,DatConclusaoTorresRealizado,DatInicioObraOutorgado_x,DatConcretagemOutorgado,DatSisTransmissaoRealizado,IdcSituacaoObra,IdcSemPrevisao,DatComissionamentoUGRealizado,DatMonitoramento_x,SigTipoGeracao,DscComercializacaoEnergia,DatInicioObraRealizado,DatMontagemRealizado,DthEnvio_x,DatConclusaoACL,DatValidadeLI,DatValidadeLP,DatValidadeLO,NomSitContratoCCD,NomSitContratoCUST,NomSituacaoContratoCUSD,NomSitContratoCCT,ValidadeAmbiental,CondicaoAmbiental,CondicaoConexao,CondicaoUso,PPA,criterio,dscriterio,criterio_novo,NomUsina,NumUgUsina,Previsao_OC,Dat_OC_obrigacao,DatMonitoramento_y,FASE,Ind_crono_norm,flagOPTeste30dias,DatPrevisaoIniciobra_y,DatInicioObraOutorgado_y,NumOperacaoUg,DthEnvio_y,DatPrevistaComercial,regra_antiga,regrapmo,Dummy,dscjustificativapmo,criterionovopmo,dsccriterionovo,regranovapmo,criterio_pmo_ts,dsccriterio_pmo_ts,criterio_pmo_ts_max_usina,dscjustificativaregra,dscjustificativaregranova,PY,CriterioNovoPMO,DscCriterioNovo,RegraNovaPMO,CriterioPMO_TS,DscCriterioPMO_TS,DscJustificativaRegra,DscJustificativaRegranova,SAS


In [80]:
merge[merge.SAS.isna() & merge.PY.notna()]

,IdeUsinaOutorga,DatCanteiroObraRealizado,DatDesvioRioRealizado,DatEnchimentoRealizado,DatConclusaoSisTransRealizado,DatPrevisaoIniciobra_x,IdcObraParalisada,IdcUsinaMonitorada,DatMontagemOutorgado,DatConcretagemRealizado,DatConclusaoTorresRealizado,DatInicioObraOutorgado_x,DatConcretagemOutorgado,DatSisTransmissaoRealizado,IdcSituacaoObra,IdcSemPrevisao,DatComissionamentoUGRealizado,DatMonitoramento_x,SigTipoGeracao,DscComercializacaoEnergia,DatInicioObraRealizado,DatMontagemRealizado,DthEnvio_x,DatConclusaoACL,DatValidadeLI,DatValidadeLP,DatValidadeLO,NomSitContratoCCD,NomSitContratoCUST,NomSituacaoContratoCUSD,NomSitContratoCCT,ValidadeAmbiental,CondicaoAmbiental,CondicaoConexao,CondicaoUso,PPA,criterio,dscriterio,criterio_novo,NomUsina,NumUgUsina,Previsao_OC,Dat_OC_obrigacao,DatMonitoramento_y,FASE,Ind_crono_norm,flagOPTeste30dias,DatPrevisaoIniciobra_y,DatInicioObraOutorgado_y,NumOperacaoUg,DthEnvio_y,DatPrevistaComercial,regra_antiga,regrapmo,Dummy,dscjustificativapmo,criterionovopmo,dsccriterionovo,regranovapmo,criterio_pmo_ts,dsccriterio_pmo_ts,criterio_pmo_ts_max_usina,dscjustificativaregra,dscjustificativaregranova,PY,CriterioNovoPMO,DscCriterioNovo,RegraNovaPMO,CriterioPMO_TS,DscCriterioPMO_TS,DscJustificativaRegra,DscJustificativaRegranova,SAS
57262,51589,NaT,NaT,NaT,NaT,2023-04-01,Não,Sim,2023-08-04,NaT,NaT,2023-02-01,2023-04-01,NaT,Não Iniciada,Não,NaT,2022-12-08 16:09:49.750,EOL,ACR,NaT,NaT,2023-01-04 12:58:25.590,NaT,NaT,2026-11-06,NaT,Não se Aplica,Não Assinado,Não se Aplica,Não Assinado,2026-11-06,LP,Sem Conexão,Sem Uso,ACR,1,Sem obras e PPA Ok,1,Ventos de Santa Luzia 15,8,2024-06-15,2024-01-01,2022-12-08,OUT,0.279832,0,2023-04-01,2023-02-01,8.0,2023-01-04 12:58:25.590,2024-01-01,2024-06-15 00:00:00,2024-06-15 00:00:00,2029-01-08,Estágio Atual das Obras,1.0,Usina sem obras com PPA = Previsão OC maior en...,2024-06-15 00:00:00,1.2,Usina sem obras com PPA = Previsão OC maior en...,1.2,Compromisso de venda de energia - PPA,Compromisso de venda de energia - PPA,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57263,51589,NaT,NaT,NaT,NaT,2023-04-01,Não,Sim,2023-08-04,NaT,NaT,2023-02-01,2023-04-01,NaT,Não Iniciada,Não,NaT,2022-12-08 16:09:49.750,EOL,ACR,NaT,NaT,2023-01-04 12:58:25.590,NaT,NaT,2026-11-06,NaT,Não se Aplica,Não Assinado,Não se Aplica,Não Assinado,2026-11-06,LP,Sem Conexão,Sem Uso,ACR,1,Sem obras e PPA Ok,1,Ventos de Santa Luzia 15,11,2024-06-15,2024-01-01,2022-12-08,OUT,0.279832,0,2023-04-01,2023-02-01,11.0,2023-01-04 12:58:25.590,2024-01-01,2024-06-15 00:00:00,2024-06-15 00:00:00,2029-01-08,Estágio Atual das Obras,1.0,Usina sem obras com PPA = Previsão OC maior en...,2024-06-15 00:00:00,1.2,Usina sem obras com PPA = Previsão OC maior en...,1.2,Compromisso de venda de energia - PPA,Compromisso de venda de energia - PPA,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57264,51589,NaT,NaT,NaT,NaT,2023-04-01,Não,Sim,2023-08-04,NaT,NaT,2023-02-01,2023-04-01,NaT,Não Iniciada,Não,NaT,2022-12-08 16:09:49.750,EOL,ACR,NaT,NaT,2023-01-04 12:58:25.590,NaT,NaT,2026-11-06,NaT,Não se Aplica,Não Assinado,Não se Aplica,Não Assinado,2026-11-06,LP,Sem Conexão,Sem Uso,ACR,1,Sem obras e PPA Ok,1,Ventos de Santa Luzia 15,2,2024-06-15,2024-01-01,2022-12-08,OUT,0.279832,0,2023-04-01,2023-02-01,2.0,2023-01-04 12:58:25.590,2024-01-01,2024-06-15 00:00:00,2024-06-15 00:00:00,2029-01-08,Estágio Atual das Obras,1.0,Usina sem obras com PPA = Previsão OC maior en...,2024-06-15 00:00:00,1.2,Usina sem obras com PPA = Previsão OC maior en...,1.2,Compromisso de venda de energia - PPA,Compromisso de venda de energia - PPA,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57265,51589,NaT,NaT,NaT,NaT,2023-04-01,Não,Sim,2023-08-04,NaT,NaT,2023-02-01,2023-04-01,NaT,Não Iniciada,Não,NaT,2022-12-08 16:09:49.750,EOL,ACR,NaT,NaT,2023-01-04 12:58:25.590,NaT,NaT,2026-11-06,NaT,Não se Aplica,Não Assinado,Não se Aplica,Não Assinado,2026-11-06,LP,Sem Conexão,Sem Uso,ACR,1,Sem obras e PPA Ok,1,Ventos de Santa Luzia 15,5,2024-06-15,2024-01-01,2022-12-08,OUT,0.279832,0,2023-04-01,2023-02-

In [ ]:
cols = ["DatUGInicioOpComerOutorgado","IdcMonitorada","IdeUsinaOutorga","DatLiberacaoSFGTeste","DatInicioOpTesteOutorgada"]
monitoramentoug = pd.read_parquet(os.path.join(directory,"vmonitoramentoug.gzip"))[cols]

In [ ]:
# vInputCronograma tbm implementado,DATASREAL (LABEL="Idempreendimento" AS Idempreendimento)
cols = ["DatRealizacaoIII","DatRealizacaoV","DatRealizacaoXIV","DatRealizacaoXII","IdeUsinaOutorga","DatRealizacaoIV","DatRealizacaoIX","DatRealizacaoXI","DatRealizacaoX","DthEnvio","DatRealizacaoVI","DatRealizacaoII","CodCeg","DatRealizacaoXIII","DatRealizacaoVII"]
rename_cols = {'DatRealizacaoIII': 'IO_real','DatRealizacaoXIV': 'Conc_Trans_REAL','DatRealizacaoXII': 'Comiss_Real','DatRealizacaoIX': 'DesvRio_real','DatRealizacaoXI': 'Ench_Real','DthEnvio': 'Data_Envio','DatRealizacaoII': 'canteiroReal','DatRealizacaoXIII': 'IOTrans_Real','DatRealizacaoVII': 'ME_Real_conc_eol'}
vcronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].drop_duplicates()[cols].rename(columns=rename_cols)
vcronograma["classe"] = vcronograma.CodCeg.str.slice(3,5)
vcronograma["CC_real"] = pd.to_datetime(pd.NA)
vcronograma.loc[vcronograma.classe == "PH","CC_real"] = vcronograma.DatRealizacaoIV
vcronograma.loc[vcronograma.classe == "CV","CC_real"] = vcronograma.DatRealizacaoV
vcronograma["ME_real"] = vcronograma.DatRealizacaoX
vcronograma.loc[vcronograma.classe == "CV","ME_real"] = vcronograma.DatRealizacaoVI

In [ ]:
sas = pd.read_excel(r'C:\Users\Vinicius\OneDrive - unb.br\0Aneel\Previsor\WORK_definicoesmonitoramento.xlsx')

In [ ]:
teste = criterios_da_previsao.drop_duplicates(subset='IdeUsinaOutorga')

In [ ]:
merge =  pd.merge(teste,sas,on="IdeUsinaOutorga",how="inner",suffixes = ('_py','_sas'))

In [ ]:
mask_cusd_cust = (merge.NomSituacaoContratoCUSD_sas == merge.NomSituacaoContratoCUSD_py) & (merge.NomSitContratoCUST_py == merge.NomSitContratoCUST_sas)

mask_envio = (merge.DthEnvio_py == merge.DthEnvio_sas)

merge[(merge.CondicaoUso_py != merge.CondicaoUso_sas) & mask_cusd_cust & mask_envio][['CondicaoUso_py','CondicaoUso_sas','NomSituacaoContratoCUSD_sas','NomSituacaoContratoCUSD_py','NomSitContratoCUST_py','NomSitContratoCUST_sas','DthEnvio_py','DthEnvio_sas']]

,CondicaoUso_py,CondicaoUso_sas,NomSituacaoContratoCUSD_sas,NomSituacaoContratoCUSD_py,NomSitContratoCUST_py,NomSitContratoCUST_sas,DthEnvio_py,DthEnvio_sas
329,Sem Uso,OK,Válido,Válido,Não Assinado,Não Assinado,2022-12-05 17:36:44.277,2022-12-05 17:36:44.277
682,Sem Uso,OK,Válido,Válido,Não Assinado,Não Assinado,2022-12-04 11:32:40.297,2022-12-04 11:32:40.297


In [ ]:
mask_cusd_cust = (merge.DatValidadeLO_py == merge.DatValidadeLO_sas) & (merge.DatValidadeLI_py == merge.DatValidadeLI_sas)

merge[(merge.CondicaoAmbiental_py != merge.CondicaoAmbiental_sas) & mask_envio][['CondicaoAmbiental_py','CondicaoAmbiental_sas','DatValidadeLO_py','DatValidadeLO_sas']]

,CondicaoAmbiental_py,CondicaoAmbiental_sas,DatValidadeLO_py,DatValidadeLO_sas


In [ ]:
merge[(merge.CondicaoAmbiental_py != merge.CondicaoAmbiental_sas) & mask_cusd_cust & mask_envio][['CondicaoAmbiental_py','CondicaoAmbiental_sas']]

,CondicaoAmbiental_py,CondicaoAmbiental_sas


In [ ]:
merge[merge.PPA_sas != merge.PPA_py]

,IdeUsinaOutorga,DatCanteiroObraRealizado,DatDesvioRioRealizado,DatEnchimentoRealizado,DatConclusaoSisTransRealizado,DatPrevisaoIniciobra_x,IdcObraParalisada_py,IdcUsinaMonitorada_py,DatMontagemOutorgado,DatConcretagemRealizado,DatConclusaoTorresRealizado,DatInicioObraOutorgado_x,DatConcretagemOutorgado,DatSisTransmissaoRealizado,IdcSituacaoObra_py,IdcSemPrevisao,DatComissionamentoUGRealizado,DatMonitoramento_x,SigTipoGeracao_py,DscComercializacaoEnergia_py,DatInicioObraRealizado,DatMontagemRealizado,DthEnvio_py,DatConclusaoACL_py,DatValidadeLI_py,DatValidadeLP_py,DatValidadeLO_py,NomSitContratoCCD_py,NomSitContratoCUST_py,NomSituacaoContratoCUSD_py,NomSitContratoCCT_py,NumOperacaoUg,DatPrevistaComercial,ValidadeAmbiental_py,CondicaoAmbiental_py,CondicaoConexao_py,CondicaoUso_py,PPA_py,criterio,dscriterio,criterio_novo,NomUsina_py,NumUgUsina,Previsao_OC,Dat_OC_obrigacao,DatMonitoramento_y,FASE,Ind_crono_norm,flagOPTeste30dias,DatPrevisaoIniciobra_y,...,SituacaoACL,PctEnergiaContratoAssinadoACL,DatConclusaoACL_sas,IdcTipoInstalacao,IdcNecessidade,NomSituacaoParAcesso,DatEmissaoParAcesso,NomSitContratoCCD_sas,DatAssinaturaContratoCCD,DatValidadeContratoCCD,NomSitContratoCCT_sas,DatAssinaturaContratoCCT,DatValidadeContratoCCT,NomSituacaoContratoCUSD_sas,DatAssinaturaContratoCUSD,DatValidadeContratoCUSD,NomSitContratoCUST_sas,DatAssinaturaContratoCUST,DatValidadeContratoCUST,DatSolicitacaoLP,DatPrevistaEmissaoLP,DatEmissaoLP,DatValidadeLP_sas,NomSituacaoLP,DatSolicitacaoLI,DatPrevistaEmissaoLI,DatEmissaoLI,DatValidadeLI_sas,NomSituacaoLI,DatSolicitacaoLO,DatPrevistaEmissaoLO,DatEmissaoLO,DatValidadeLO_sas,NomSituacaoLO,IdcTipoEntrave,DscSolucao,DscObservacaoAcesso,DscObservacaomercado,DscObservacaoAmbiental,ValidadeAmbiental_sas,CondicaoAmbiental_sas,CondicaoConexao_sas,CondicaoUso_sas,PPA_sas,DscComercializacaoEnergia_sas,DscSistema,DatPrevistaComercialRapeelMX,DscJustificativaPrevisao,DscObservacaoUG,DatEmissaoAto
